# Structural Balance & GNN Experiment

**Hypothesis:** MagNet outperforms GCN when graphs are structurally unbalanced.

**Unbalance score:** `1 - rho(A_mag) / rho(A_sym)` where:
- `A_sym` = symmetric (undirected) adjacency
- `A_mag` = Hermitian (magnetic) adjacency with phase encoding direction

Higher score = more directional information lost when symmetrizing = more potential for MagNet advantage.

In [ ]:
# Cell 1: Setup & Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.linalg import eigvalsh
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

print("Setup complete!")

In [ ]:
# Cell 2: Matrix Construction

def build_matrices(edge_index, num_nodes, q=0.1):
    """
    Build normalized adjacency matrices for GCN and MagNet.
    
    Args:
        edge_index: [2, E] tensor of directed edges
        num_nodes: number of nodes
        q: phase parameter (0.1 gives good real/imag balance)
    
    Returns:
        A_gcn: symmetric normalized adjacency
        A_mag: Hermitian normalized adjacency
    """
    # Symmetric adjacency (treat all edges as undirected)
    A_sym = torch.zeros(num_nodes, num_nodes)
    for i in range(edge_index.shape[1]):
        u, v = edge_index[0, i].item(), edge_index[1, i].item()
        A_sym[u, v] = 1.0
        A_sym[v, u] = 1.0
    
    # Add self-loops
    A_sym = A_sym + torch.eye(num_nodes)
    
    # Degree normalization
    D = A_sym.sum(dim=1)
    D_inv_sqrt = torch.zeros(num_nodes)
    D_inv_sqrt[D > 0] = 1.0 / torch.sqrt(D[D > 0])
    D_mat = torch.diag(D_inv_sqrt)
    
    # GCN: symmetric normalized
    A_gcn = D_mat @ A_sym @ D_mat
    
    # MagNet: Hermitian with phase encoding direction
    A_mag = torch.eye(num_nodes, dtype=torch.complex64)  # Self-loops
    phase = 2 * np.pi * q
    for i in range(edge_index.shape[1]):
        u, v = edge_index[0, i].item(), edge_index[1, i].item()
        A_mag[u, v] = np.exp(1j * phase)   # u -> v
        A_mag[v, u] = np.exp(-1j * phase)  # Hermitian conjugate
    
    # Same normalization
    D_mat_c = D_mat.to(torch.complex64)
    A_mag = D_mat_c @ A_mag @ D_mat_c
    
    return A_gcn, A_mag


def unbalance_score(edge_index, num_nodes, q=0.1):
    """Compute structural unbalance: 1 - rho(A_mag)/rho(A_sym)."""
    A_gcn, A_mag = build_matrices(edge_index, num_nodes, q)
    
    rho_gcn = np.max(np.abs(np.linalg.eigvalsh(A_gcn.numpy())))
    rho_mag = np.max(np.abs(np.linalg.eigvals(A_mag.numpy())))
    
    if rho_gcn == 0:
        return 0.0
    return 1 - rho_mag / rho_gcn


# Test
print("Testing matrix construction...")
test_edges = torch.tensor([[0,1,2],[1,2,0]], dtype=torch.long)
A_g, A_m = build_matrices(test_edges, 3, q=0.1)
print(f"GCN matrix shape: {A_g.shape}")
print(f"MagNet matrix shape: {A_m.shape}")
print(f"Unbalance score: {unbalance_score(test_edges, 3):.4f}")

In [ ]:
# Cell 3: Graph Generation

def generate_sbm(n, k, p_in, p_out, balanced=True):
    """
    Generate directed stochastic block model.
    
    balanced=True: consistent edge directions (structurally balanced)
    balanced=False: random directions (creates odd cycles, unbalanced)
    """
    labels = torch.tensor([i % k for i in range(n)])
    edges = []
    
    for i in range(n):
        for j in range(i + 1, n):
            p = p_in if labels[i] == labels[j] else p_out
            if np.random.rand() < p:
                if balanced:
                    edges.append([i, j])  # Consistent: low -> high
                else:
                    # Random direction creates odd cycles
                    if np.random.rand() < 0.5:
                        edges.append([i, j])
                    else:
                        edges.append([j, i])
    
    if len(edges) == 0:
        edges = [[0, 1]]
    
    return torch.tensor(edges, dtype=torch.long).t(), labels


# Test generators
print("Testing graph generators...")
for name, balanced in [("Balanced", True), ("Unbalanced", False)]:
    ei, lab = generate_sbm(50, 3, 0.3, 0.1, balanced)
    score = unbalance_score(ei, 50)
    print(f"{name} SBM: {ei.shape[1]} edges, unbalance={score:.4f}")

In [ ]:
# Cell 4: GNN Models

class GCN(nn.Module):
    """Standard 2-layer Graph Convolutional Network."""
    
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x, A):
        # Layer 1: aggregate -> transform -> activate
        x = A @ x
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        # Layer 2: aggregate -> transform
        x = A @ x
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


class MagNet(nn.Module):
    """
    Magnetic Graph Neural Network.
    
    Uses complex adjacency to encode edge direction.
    Processes real and imaginary channels separately then combines.
    """
    
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        # Separate transforms for real and imaginary parts
        self.fc1_re = nn.Linear(in_dim, hidden_dim)
        self.fc1_im = nn.Linear(in_dim, hidden_dim)
        self.fc2_re = nn.Linear(hidden_dim, out_dim)
        self.fc2_im = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x, A_mag):
        # Layer 1: complex aggregation
        x_c = x.to(torch.complex64)
        h = A_mag @ x_c
        
        # Process real and imaginary separately, then combine
        h_re = F.relu(self.fc1_re(h.real))
        h_im = F.relu(self.fc1_im(h.imag))
        h = h_re + h_im
        h = F.dropout(h, p=0.5, training=self.training)
        
        # Layer 2: complex aggregation
        h_c = h.to(torch.complex64)
        h = A_mag @ h_c
        
        out = self.fc2_re(h.real) + self.fc2_im(h.imag)
        
        return F.log_softmax(out, dim=1)


print("Models defined!")

In [ ]:
# Cell 5: Training & Evaluation

def train_eval(model, x, A, labels, train_mask, test_mask, epochs=200):
    """Train model and return test accuracy."""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    model.train()
    for _ in range(epochs):
        optimizer.zero_grad()
        out = model(x, A)
        loss = F.nll_loss(out[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        pred = model(x, A).argmax(dim=1)
        acc = (pred[test_mask] == labels[test_mask]).float().mean()
    
    return acc.item()


print("Training function ready!")

In [ ]:
# Cell 6: Main Experiment

def run_experiment(num_trials=10, q=0.1):
    """Run full experiment comparing GCN vs MagNet."""
    results = []
    n, k = 100, 4
    
    for name, balanced in [("Balanced", True), ("Unbalanced", False)]:
        print(f"\n{name}:")
        for trial in range(num_trials):
            # Generate graph
            edge_index, labels = generate_sbm(n, k, 0.3, 0.05, balanced)
            A_gcn, A_mag = build_matrices(edge_index, n, q)
            
            # Features and masks
            x = torch.randn(n, 16)
            perm = torch.randperm(n)
            train_mask = torch.zeros(n, dtype=torch.bool)
            train_mask[perm[:60]] = True
            test_mask = ~train_mask
            
            # Compute unbalance score
            score = unbalance_score(edge_index, n, q)
            
            # Train GCN
            gcn = GCN(16, 32, k)
            acc_gcn = train_eval(gcn, x, A_gcn, labels, train_mask, test_mask)
            
            # Train MagNet
            magnet = MagNet(16, 32, k)
            acc_mag = train_eval(magnet, x, A_mag, labels, train_mask, test_mask)
            
            delta = acc_mag - acc_gcn
            
            results.append({
                'config': name,
                'trial': trial,
                'unbalance': score,
                'gcn': acc_gcn,
                'magnet': acc_mag,
                'delta': delta
            })
            
            print(f"  {trial}: unbal={score:.3f}, GCN={acc_gcn:.3f}, MagNet={acc_mag:.3f}, d={delta:+.3f}")
    
    return pd.DataFrame(results)


# Run experiment
print("Starting experiment...")
df = run_experiment(num_trials=10, q=0.1)
print("\nExperiment complete!")

In [ ]:
# Cell 7: Visualization

# Summary statistics
print("Summary by configuration:")
summary = df.groupby('config')[['unbalance', 'gcn', 'magnet', 'delta']].mean()
print(summary.round(3))
print()

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left: Scatter of unbalance vs delta
colors = {'Balanced': 'blue', 'Unbalanced': 'red'}
for cfg in df['config'].unique():
    sub = df[df['config'] == cfg]
    ax1.scatter(sub['unbalance'], sub['delta'], label=cfg,
               c=colors[cfg], alpha=0.7, s=80)
ax1.axhline(0, color='k', linestyle='--', alpha=0.3)
ax1.set_xlabel('Unbalance Score')
ax1.set_ylabel('MagNet - GCN Accuracy')
ax1.set_title('Structural Unbalance vs Performance Gap')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Bar comparison
x_pos = np.arange(2)
ax2.bar(x_pos - 0.175, summary['gcn'], 0.35, label='GCN', color='steelblue')
ax2.bar(x_pos + 0.175, summary['magnet'], 0.35, label='MagNet', color='coral')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(summary.index)
ax2.set_ylabel('Accuracy')
ax2.set_title('Model Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('structural_balance_results.png', dpi=150)
plt.show()

print("\nFigure saved as 'structural_balance_results.png'")

In [ ]:
# Cell 8: Statistical Analysis

# Correlation
corr, p_corr = stats.pearsonr(df['unbalance'], df['delta'])
print(f"Correlation (unbalance vs delta): r={corr:.3f}, p={p_corr:.4f}")

# T-test between groups
bal = df[df['config'] == 'Balanced']['delta']
unbal = df[df['config'] == 'Unbalanced']['delta']
t, p = stats.ttest_ind(unbal, bal)
print(f"T-test: t={t:.3f}, p={p:.4f}")

# Conclusion
print("\n" + "="*50)
print("CONCLUSION:")
if corr > 0.2 and unbal.mean() > bal.mean():
    print("SUPPORTS hypothesis: MagNet advantage on unbalanced graphs")
elif corr < -0.2:
    print("CONTRADICTS hypothesis")
else:
    print("INCONCLUSIVE - need more trials or different setup")
print("="*50)

## Notes

**Key parameters:**
- `q=0.1`: Phase parameter for magnetic adjacency (good real/imag balance)
- `n=100, k=4`: 100 nodes, 4 classes in SBM
- `p_in=0.3, p_out=0.05`: Edge probabilities

**If results are inconclusive:**
- Try different `q` values (0.05, 0.15, 0.2)
- Increase `num_trials`
- Try larger graphs or different SBM parameters